In [2]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [7]:
sub9455 = pd.read_csv('../submit/tmp/20190912_0112__CV0-9455725346531509__feature655.csv')
sub9467 = pd.read_csv('../submit/lb/20190909_2324__CV0-9467296784440689__feature552.csv')

In [8]:
COLUMN_TARGET = 'isFraud'
sub9455[COLUMN_TARGET] = sub9455[COLUMN_TARGET].values*0.5 + sub9467[COLUMN_TARGET].values*0.5
sub9455[COLUMN_TARGET].head()

0    0.000529
1    0.001495
2    0.001028
3    0.001014
4    0.000949
Name: isFraud, dtype: float64

In [13]:
np.corrcoef(sub9455[COLUMN_TARGET].values, sub9467[COLUMN_TARGET].values)

array([[1.        , 0.99487246],
       [0.99487246, 1.        ]])

In [10]:
sub9455.to_csv('../submit/lb/20190912_0820__CV0-948821__lr0-01_esr100_nl255.csv', index=False)